# SAP HANA Cloud Setup - User & Schema Management

This notebook helps you manage your SAP HANA Cloud environment for the Narrow AI best practices projects.

## What this notebook does:
1. **CREATE**: Schema and user with all necessary privileges including PAL
2. **DELETE**: Clean up schemas and users when needed
3. **VERIFY**: Test connections and privileges

## Prerequisites:
- You must run this notebook using an admin account (e.g., DBADMIN)
- Your `.env` file should contain admin credentials

## Workflow:
1. Run this notebook with DBADMIN credentials
2. Create schema and user
3. Update `.env` file with new user credentials
4. Use new user for all project notebooks

## Install Required Packages

In [ ]:
%pip install hana-ml python-dotenv

## Load Environment Variables and Connect

Make sure your `.env` file contains **admin credentials** (e.g., DBADMIN).

In [1]:
from hdbcli import dbapi
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get admin connection details
hana_address = os.getenv('hana_address', '<your_hana_address>')
hana_port = int(os.getenv('hana_port', 443))
admin_user = os.getenv('hana_user', 'DBADMIN')  # Must be admin user
admin_password = os.getenv('hana_password', '<your_password>')
hana_schema = os.getenv('HANA_SCHEMA', 'DBADMIN') # Default schema or your specific one

print(f"Connecting to HANA as admin user: {admin_user}")

# Connect to HANA
conn = dbapi.connect(
    address=hana_address,
    port=hana_port,
    user=admin_user,
    password=admin_password,
    encrypt=True,
    sslValidateCertificate=False
)

print("✓ Connected successfully!")

Connecting to HANA as admin user: DBADMIN
✓ Connected successfully!


---

# Part 1: CREATE - Schema and User

Run the cells below to create a new schema and user.

## Configuration - Set Your Schema and User Details

**⚠️ IMPORTANT: Modify these values before running!**

In [2]:
# ========================================
# CONFIGURATION - CHANGE THESE VALUES!
# ========================================

# Schema to create for projects
PROJECT_SCHEMA = hana_schema

# New user credentials
NEW_USER_NAME = 'PAL_ADMIN'
NEW_USER_PASSWORD = 'PAL_ADNINa123!'  # Use a strong password!

print(f"Schema to create: {PROJECT_SCHEMA}")
print(f"User to create: {NEW_USER_NAME}")
print(f"Password: {'*' * len(NEW_USER_PASSWORD)}")

Schema to create: BTP_AI_BP
User to create: PAL_ADMIN
Password: **************


## Step 1: Create Schema

In [3]:
cursor = conn.cursor()

try:
    print(f"Creating schema: {PROJECT_SCHEMA}\n")
    
    # Check if schema already exists
    cursor.execute(f"SELECT SCHEMA_NAME FROM SYS.SCHEMAS WHERE SCHEMA_NAME = '{PROJECT_SCHEMA}'")
    schema_exists = cursor.fetchone()
    
    if schema_exists:
        print(f"⚠️  Schema '{PROJECT_SCHEMA}' already exists. Skipping creation.\n")
    else:
        # Create the schema
        cursor.execute(f"CREATE SCHEMA {PROJECT_SCHEMA}")
        print(f"✓ Schema '{PROJECT_SCHEMA}' created successfully!\n")
    
    print("Current user-defined schemas:")
    cursor.execute("SELECT SCHEMA_NAME FROM SYS.SCHEMAS WHERE SCHEMA_NAME NOT LIKE '_SYS%' AND SCHEMA_NAME NOT LIKE 'SAP_%' ORDER BY SCHEMA_NAME")
    schemas = cursor.fetchall()
    for schema in schemas:
        marker = " ← TARGET" if schema[0] == PROJECT_SCHEMA else ""
        print(f"  - {schema[0]}{marker}")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    cursor.close()

Creating schema: BTP_AI_BP

✓ Schema 'BTP_AI_BP' created successfully!

Current user-defined schemas:
  - 190CF386E29247B5B828004BF73D50B7
  - 190CF386E29247B5B828004BF73D50B7#DI
  - 190CF386E29247B5B828004BF73D50B7#OO
  - 190CF386E29247B5B828004BF73D50B7_CIBIP0083825PXUYITHB4JOX9_DT
  - 190CF386E29247B5B828004BF73D50B7_CIBIP0083825PXUYITHB4JOX9_RT
  - 1FFE88C65B734B8EB96789FFD815476B
  - 1FFE88C65B734B8EB96789FFD815476B#DI
  - 1FFE88C65B734B8EB96789FFD815476B#OO
  - 1FFE88C65B734B8EB96789FFD815476B_DFNJXD0DO3NY315XR66B9KSXY_DT
  - 1FFE88C65B734B8EB96789FFD815476B_DFNJXD0DO3NY315XR66B9KSXY_RT
  - 1FFE88C65B734B8EB96789FFD815476B_E74ERDF61ISMJ3XGUDVQ2SATR_DT
  - 1FFE88C65B734B8EB96789FFD815476B_E74ERDF61ISMJ3XGUDVQ2SATR_RT
  - 6009E0FCFEF04E54A6D29A6FD0B2D692
  - 6009E0FCFEF04E54A6D29A6FD0B2D692#DI
  - 6009E0FCFEF04E54A6D29A6FD0B2D692#OO
  - 6009E0FCFEF04E54A6D29A6FD0B2D692_7OWFTCKJ8C5EATEKWBCRYGQ3I_DT
  - 6009E0FCFEF04E54A6D29A6FD0B2D692_7OWFTCKJ8C5EATEKWBCRYGQ3I_RT
  - 6009E0FCFEF04E5

## Step 2: Create User with Full Privileges

In [4]:
cursor = conn.cursor()

try:
    print(f"Creating user: {NEW_USER_NAME}\n")
    
    # Check if user already exists
    cursor.execute(f"SELECT USER_NAME FROM SYS.USERS WHERE USER_NAME = '{NEW_USER_NAME}'")
    user_exists = cursor.fetchone()
    
    if user_exists:
        print(f"⚠️  User '{NEW_USER_NAME}' already exists. Skipping creation.")
        print("Proceeding to grant privileges...\n")
    else:
        # Create new user
        cursor.execute(f"CREATE USER {NEW_USER_NAME} PASSWORD \"{NEW_USER_PASSWORD}\" NO FORCE_FIRST_PASSWORD_CHANGE")
        print(f"✓ User '{NEW_USER_NAME}' created successfully!\n")
    
    # Grant privileges
    print("Granting privileges...\n")
    
    # 1. Schema privileges (MOST IMPORTANT)
    cursor.execute(f"GRANT ALL PRIVILEGES ON SCHEMA {PROJECT_SCHEMA} TO {NEW_USER_NAME}")
    print(f"✓ Granted ALL PRIVILEGES on schema {PROJECT_SCHEMA}")
    
    # 2. PAL (Predictive Analysis Library) privileges
    cursor.execute(f"GRANT AFL__SYS_AFL_AFLPAL_EXECUTE TO {NEW_USER_NAME}")
    print("✓ Granted AFL__SYS_AFL_AFLPAL_EXECUTE")
    
    # Try to grant specific PAL functions (may fail with insufficient privilege)
    try:
        cursor.execute(f"GRANT EXECUTE ON _SYS_AFL.PAL_UNIVARIATE_ANALYSIS TO {NEW_USER_NAME}")
        print("✓ Granted EXECUTE on PAL_UNIVARIATE_ANALYSIS")
    except Exception as e:
        print(f"⚠️  Could not grant PAL_UNIVARIATE_ANALYSIS (insufficient privilege)")
        print(f"   This is OK - the main PAL privilege was already granted")
    
    # 3. Object creation privileges (may fail)
    try:
        cursor.execute(f"GRANT CREATE ANY TO {NEW_USER_NAME}")
        print("✓ Granted CREATE ANY")
    except Exception as e:
        print(f"⚠️  Could not grant CREATE ANY (insufficient privilege)")
        print(f"   This is OK - schema privileges are sufficient for creating tables")
    
    # 4. System views access
    try:
        cursor.execute(f"GRANT SELECT ON SCHEMA _SYS_BIC TO {NEW_USER_NAME}")
        print("✓ Granted SELECT on _SYS_BIC")
    except:
        print("⚠️  Could not grant SELECT on _SYS_BIC")
    
    try:
        cursor.execute(f"GRANT SELECT ON SCHEMA SYS TO {NEW_USER_NAME}")
        print("✓ Granted SELECT on SYS")
    except:
        print("⚠️  Could not grant SELECT on SYS")
    
    # 5. Additional privileges for ML/PAL operations
    try:
        cursor.execute(f"GRANT EXECUTE ON SCHEMA _SYS_AFL TO {NEW_USER_NAME}")
        print("✓ Granted EXECUTE on _SYS_AFL")
    except:
        print("⚠️  Could not grant EXECUTE on _SYS_AFL")
    
    print(f"\n✅ User '{NEW_USER_NAME}' is ready to use!")
    print(f"\n📋 Summary of granted privileges:")
    print(f"   ✓ ALL PRIVILEGES on schema {PROJECT_SCHEMA} (can create/modify tables)")
    print(f"   ✓ AFL PAL EXECUTE (can use PAL/ML functions)")
    
except Exception as e:
    print(f"❌ Critical Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    cursor.close()

Creating user: PAL_ADMIN

✓ User 'PAL_ADMIN' created successfully!

Granting privileges...

✓ Granted ALL PRIVILEGES on schema BTP_AI_BP
✓ Granted AFL__SYS_AFL_AFLPAL_EXECUTE
⚠️  Could not grant PAL_UNIVARIATE_ANALYSIS (insufficient privilege)
   This is OK - the main PAL privilege was already granted
⚠️  Could not grant CREATE ANY (insufficient privilege)
   This is OK - schema privileges are sufficient for creating tables
⚠️  Could not grant SELECT on _SYS_BIC
⚠️  Could not grant SELECT on SYS
⚠️  Could not grant EXECUTE on _SYS_AFL

✅ User 'PAL_ADMIN' is ready to use!

📋 Summary of granted privileges:
   ✓ ALL PRIVILEGES on schema BTP_AI_BP (can create/modify tables)
   ✓ AFL PAL EXECUTE (can use PAL/ML functions)


## Step 3: Verify User Privileges

In [5]:
cursor = conn.cursor()

try:
    print(f"Checking privileges for {NEW_USER_NAME}:\n")
    
    # Check granted privileges
    cursor.execute(f"""
        SELECT PRIVILEGE, OBJECT_TYPE, SCHEMA_NAME, OBJECT_NAME
        FROM SYS.GRANTED_PRIVILEGES
        WHERE GRANTEE = '{NEW_USER_NAME}'
        ORDER BY PRIVILEGE
    """)
    
    privileges = cursor.fetchall()
    
    if privileges:
        print(f"Total privileges granted: {len(privileges)}\n")
        print("Key privileges:")
        
        pal_privs = [p for p in privileges if 'AFL' in str(p) or 'PAL' in str(p)]
        schema_privs = [p for p in privileges if PROJECT_SCHEMA in str(p)]
        
        print(f"\nPAL-related privileges: {len(pal_privs)}")
        for priv in pal_privs[:5]:  # Show first 5
            print(f"  - {priv[0]} on {priv[1]}")
        
        print(f"\nSchema '{PROJECT_SCHEMA}' privileges: {len(schema_privs)}")
        for priv in schema_privs[:5]:  # Show first 5
            print(f"  - {priv[0]} on {priv[1]}")
    else:
        print("⚠️  No privileges found. There may have been an error.")
        
except Exception as e:
    print(f"❌ Error: {e}")
finally:
    cursor.close()

Checking privileges for PAL_ADMIN:

Total privileges granted: 19

Key privileges:

PAL-related privileges: 1
  - CREATE ANY on SCHEMA

Schema 'BTP_AI_BP' privileges: 18
  - ALTER on SCHEMA
  - CLIENTSIDE ENCRYPTION COLUMN KEY ADMIN on SCHEMA
  - CREATE ANY on SCHEMA
  - CREATE OBJECT STRUCTURED PRIVILEGE on SCHEMA
  - CREATE TEMPORARY TABLE on SCHEMA


## Step 3.5: Check Script Server Status (Required for PAL)

⚠️ **IMPORTANT**: PAL (Predictive Analysis Library) requires the Script Server to be enabled.

SAP HANA Cloud instances need **minimum 3 vCPUs** and Script Server must be enabled to use PAL functions.

### How to Enable Script Server (Official SAP Guide)

⚠️ **IMPORTANT**:
- **Script Server CANNOT be controlled via SQL/Python code** ❌
- It's a **system-level configuration**, not a database setting
- You must use **BTP Cockpit GUI** or contact **SAP Support** ✅

#### Prerequisites:
- SAP HANA Cloud instance with **minimum 3 vCPUs**
- Trial instances do NOT support Script Server

#### Method 1: BTP Cockpit (Recommended)

1. **Open SAP BTP Cockpit**
   - Navigate to your SAP BTP Cockpit
   - Go to your Subaccount

2. **Access HANA Cloud Central**
   - Click on **SAP HANA Cloud** 
   - Find your instance

3. **Edit Instance Configuration**
   - Click on instance **Actions (⋯)** 
   - Select **"Manage Configuration"**
   - Go to **"Advanced Settings"** or **"Additional Features"** tab
   - Look for **"Script Server"** option
   - Enable Script Server checkbox
   - Save configuration

4. **Restart Instance** (if required)
   - Some changes require instance restart
   - Follow prompts to restart

#### Method 2: Contact SAP Support

If Script Server option is not available in BTP Cockpit:

1. Open support ticket on component **HAN-CLS-HC**
2. Request Script Server activation
3. Provide instance details (Instance ID, vCPUs)
4. Wait for SAP Support to enable it

#### Why can't we use code?

Script Server is a system-level service that runs alongside your HANA database:
- It requires instance-level configuration changes
- Similar to enabling/disabling the database itself
- Needs infrastructure-level permissions beyond database users
- SAP restricts this to BTP Cockpit for security and stability

#### Important Notes:

- **Cost**: No additional cost for Script Server itself, but requires 3+ vCPU instance (which costs more than smaller instances)
- **Trial Limitations**: Script Server NOT available in trial accounts
- **Restart Required**: Instance may need restart after enabling
- **No SQL/API control**: This is by design for system security

#### Official Documentation:

- 📖 [SAP KBA 3216010 - Enable/Disable Script Server](https://userapps.support.sap.com/sap/support/knowledge/en/3216010) (requires SAP login)
- 📖 [SAP HANA Cloud - Script Server Configuration](https://help.sap.com/docs/SAP_DATASPHERE/9f804b8efa8043539289f42f372c4862/287194276a7d4d778ec98fdde5f61335.html)
- 📖 [Getting Started with PAL and APL](https://community.sap.com/t5/technology-blog-posts-by-sap/getting-started-with-sap-hana-cloud-pal-and-apl/ba-p/13482157)

In [7]:
cursor = conn.cursor()

try:
    print("Checking Script Server status...\n")
    
    # Check if scriptserver is running
    cursor.execute("""
        SELECT 
            HOST,
            SERVICE_NAME,
            ACTIVE_STATUS,
            PROCESS_ID,
            PORT,
            COORDINATOR_TYPE
        FROM M_SERVICES 
        WHERE SERVICE_NAME = 'scriptserver'
        ORDER BY HOST
    """)
    
    scriptservers = cursor.fetchall()
    
    if scriptservers:
        print("Script Server instances found:")
        for ss in scriptservers:
            host, service, status, pid, port, coord = ss
            status_icon = "✅" if status == "YES" else "❌"
            print(f"  {status_icon} Host: {host}")
            print(f"     Service: {service}")
            print(f"     Status: {status}")
            print(f"     Process ID: {pid}")
            print(f"     Port: {port}")
            print(f"     Coordinator: {coord}")
            print()
        
        # Check if at least one is active
        active_count = sum(1 for ss in scriptservers if ss[2] == "YES")
        
        if active_count > 0:
            print(f"✅ Script Server is ACTIVE ({active_count} instance(s) running)")
            print("   PAL functions are available!")
        else:
            print("❌ Script Server is NOT ACTIVE")
            print("\n⚠️  WARNING: PAL functions will NOT work without Script Server!")
            print("\n📖 How to enable Script Server:")
            print("   1. SAP HANA Cloud instance needs minimum 3 vCPUs")
            print("   2. Contact SAP Support or your administrator")
            print("   3. Reference: SAP KBA 3216010 - Enable/Disable Script Server")
            print("\n🔗 More info:")
            print("   - https://community.sap.com/t5/technology-blog-posts-by-sap/getting-started-with-sap-hana-cloud-pal-and-apl/ba-p/13482157")
    else:
        print("❌ No Script Server instances found")
        print("\n⚠️  WARNING: Script Server is not configured!")
        print("\nYour HANA Cloud instance may not support PAL.")
        print("\n📖 Requirements:")
        print("   - SAP HANA Cloud instance with minimum 3 vCPUs")
        print("   - Script Server must be enabled (contact SAP Support)")
        print("\n🔗 Documentation:")
        print("   - SAP KBA 3216010: How-To Enable/Disable Script Server")
        print("   - https://help.sap.com/docs/hana-cloud/sap-hana-cloud-getting-started-guide/using-machine-learning-libraries-apl-and-pal-in-sap-hana-cloud-sap-hana-database")
    
except Exception as e:
    print(f"❌ Error checking Script Server: {e}")
    print("\nThis might indicate permission issues or the view is not accessible.")
finally:
    cursor.close()

Checking Script Server status...

❌ No Script Server instances found

⚠️  WARNING: Script Server is not configured!

Your HANA Cloud instance may not support PAL.

📖 Requirements:
   - SAP HANA Cloud instance with minimum 3 vCPUs
   - Script Server must be enabled (contact SAP Support)

🔗 Documentation:
   - SAP KBA 3216010: How-To Enable/Disable Script Server
   - https://help.sap.com/docs/hana-cloud/sap-hana-cloud-getting-started-guide/using-machine-learning-libraries-apl-and-pal-in-sap-hana-cloud-sap-hana-database


## Step 4: Test Connection with New User

In [6]:
# Test connection with new user
try:
    print(f"Testing connection with new user: {NEW_USER_NAME}\n")
    
    test_conn = dbapi.connect(
        address=hana_address,
        port=hana_port,
        user=NEW_USER_NAME,
        password=NEW_USER_PASSWORD,
        encrypt=True,
        sslValidateCertificate=False
    )
    
    test_cursor = test_conn.cursor()
    test_cursor.execute("SELECT CURRENT_USER, CURRENT_SCHEMA FROM DUMMY")
    result = test_cursor.fetchone()
    
    print(f"✓ Connection successful!")
    print(f"  Current User: {result[0]}")
    print(f"  Current Schema: {result[1] if result[1] else 'None (will use default)'}")
    
    test_cursor.close()
    test_conn.close()
    
    print("\n✅ New user is working correctly!")
    
except Exception as e:
    print(f"❌ Connection test failed: {e}")
    print("\nPlease check:")
    print("1. User was created successfully")
    print("2. Password is correct")
    print("3. User has necessary privileges")

Testing connection with new user: PAL_ADMIN

✓ Connection successful!
  Current User: PAL_ADMIN
  Current Schema: PAL_ADMIN

✅ New user is working correctly!


## Step 5: Update Your .env File

Copy the configuration below to your `.env` file:

In [ ]:
print("="*60)
print("ADD THESE LINES TO YOUR .env FILE:")
print("="*60)
print(f"""\nhana_address={hana_address}
hana_port={hana_port}
hana_user={NEW_USER_NAME}
hana_password={NEW_USER_PASSWORD}
hana_encrypt=True
HANA_SCHEMA={PROJECT_SCHEMA}
""")
print("="*60)
print("\n⚠️  IMPORTANT: Keep your .env file secure and never commit it to git!")
print("\nNext steps:")
print("1. Update your .env file with the credentials above")
print("2. Restart your Jupyter kernel")
print("3. Run any project notebook (e.g., time-series-forecasting, classification, etc.)")
print("4. The notebooks will now connect using the new user account")

---

# Part 2: DELETE - Cleanup Resources

⚠️ **WARNING: The cells below will permanently DELETE users and schemas!**

Run these cells only when you want to clean up and remove resources.

## Delete User

⚠️ **This will permanently delete the user account!**

In [ ]:
# User to delete - CHANGE THIS!
USER_TO_DELETE = 'PROJECT_USER'  # Set to the user you want to delete

cursor = conn.cursor()

try:
    print(f"⚠️  WARNING: About to delete user '{USER_TO_DELETE}'\n")
    
    # Check if user exists
    cursor.execute(f"SELECT USER_NAME FROM SYS.USERS WHERE USER_NAME = '{USER_TO_DELETE}'")
    user_exists = cursor.fetchone()
    
    if not user_exists:
        print(f"User '{USER_TO_DELETE}' does not exist.")
    else:
        # Delete the user (CASCADE removes all objects owned by user)
        cursor.execute(f"DROP USER {USER_TO_DELETE} CASCADE")
        print(f"✓ User '{USER_TO_DELETE}' deleted successfully!")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    cursor.close()

## Delete Schema

⚠️ **This will permanently delete the schema and ALL tables/objects inside it!**

In [ ]:
# Schema to delete - CHANGE THIS!
SCHEMA_TO_DELETE = 'BTP_AI_BP'  # Set to the schema you want to delete

cursor = conn.cursor()

try:
    print(f"⚠️  WARNING: About to delete schema '{SCHEMA_TO_DELETE}'\n")
    
    # Check if schema exists
    cursor.execute(f"SELECT SCHEMA_NAME FROM SYS.SCHEMAS WHERE SCHEMA_NAME = '{SCHEMA_TO_DELETE}'")
    schema_exists = cursor.fetchone()
    
    if not schema_exists:
        print(f"Schema '{SCHEMA_TO_DELETE}' does not exist.")
    else:
        # List tables in the schema before deletion
        cursor.execute(f"""
            SELECT TABLE_NAME 
            FROM SYS.TABLES 
            WHERE SCHEMA_NAME = '{SCHEMA_TO_DELETE}'
        """)
        tables = cursor.fetchall()
        
        if tables:
            print(f"Tables in '{SCHEMA_TO_DELETE}' that will be deleted:")
            for table in tables:
                print(f"  - {table[0]}")
            print()
        else:
            print(f"Schema '{SCHEMA_TO_DELETE}' is empty.\n")
        
        # Delete the schema (CASCADE deletes all objects inside)
        cursor.execute(f"DROP SCHEMA {SCHEMA_TO_DELETE} CASCADE")
        print(f"✓ Schema '{SCHEMA_TO_DELETE}' deleted successfully!")
        
        # List remaining schemas
        print("\nRemaining user-defined schemas:")
        cursor.execute("SELECT SCHEMA_NAME FROM SYS.SCHEMAS WHERE SCHEMA_NAME NOT LIKE '_SYS%' AND SCHEMA_NAME NOT LIKE 'SAP_%' ORDER BY SCHEMA_NAME")
        schemas = cursor.fetchall()
        for schema in schemas:
            print(f"  - {schema[0]}")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    cursor.close()

---

# Final Cleanup

Close the admin connection when done.

In [ ]:
try:
    conn.close()
    print("✓ Admin connection closed.")
except:
    pass

---

## Summary

### CREATE Functions:
- ✅ Create schema for projects
- ✅ Create user with full privileges
- ✅ Grant PAL (Predictive Analysis Library) access
- ✅ Test connection
- ✅ Generate `.env` configuration

### DELETE Functions:
- ✅ Delete users (with CASCADE)
- ✅ Delete schemas (with all tables)
- ✅ List contents before deletion

---

## Available Best Practice Projects:

After setup, you can run any of these project notebooks:
- `anomaly-detection/`
- `classification/`
- `clustering/`
- `regression/`
- `time-series-forecasting/`

All projects will use the user credentials configured in your `.env` file!